In [41]:
import argparse

import torch
import uuid

import numpy as np

from torch_geometric.nn import Node2Vec
from torch_geometric.utils import erdos_renyi_graph

import os
import csv

import gph
import giotto

ModuleNotFoundError: No module named 'giotto'

In [39]:
## import files
d = 5
n = 10
l = 10 
c = 5

filename = 'er'
filename += '-c'+str(c)
filename += '-d'+ str(d)
filename += '-l'+str(l)
filename += '-n'+ str(n)

## get filenames according to filter
path = '../rawos'
files = []
for i in os.listdir(path):
    if os.path.isfile(os.path.join(path,i)) and filename in i:
        files.append(i)
        
files

['er-c5-d5-l10-n10-3ab3b40f452544febe3662e98bc3cdfe.tsv',
 'er-c5-d5-l10-n10-3eb59b182eb3460081807487e0a76f35.tsv']

In [27]:
## load embedding 
emb = np.loadtxt(path+'/'+files[0], delimiter="\t")
print(emb.shape)

(200, 5)


In [29]:
## compute PH
ph= gph.ripser_parallel(X = emb, maxdim =2 , coeff = 2, metric = 'euclidean',thresh=np.inf )

In [37]:
len(ph['dgms'])

3

In [40]:
## compute distances
